<center>
<img src="https://habrastorage.org/files/fd4/502/43d/fd450243dd604b81b9713213a247aa20.jpg">
    
## [mlcourse.ai](https://mlcourse.ai) – Open Machine Learning Course 
Author: [Yury Kashnitskiy](https://yorko.github.io) (@yorko). This material is subject to the terms and conditions of the [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license. Free use is permitted for any non-commercial purpose.

## <center> Assignment 4. Sarcasm detection with logistic regression
    
We'll be using the dataset from the [paper](https://arxiv.org/abs/1704.05579) "A Large Self-Annotated Corpus for Sarcasm" with >1mln comments from Reddit, labeled as either sarcastic or not. A processed version can be found on Kaggle in a form of a [Kaggle Dataset](https://www.kaggle.com/danofer/sarcasm).

Sarcasm detection is easy. 
<img src="https://habrastorage.org/webt/1f/0d/ta/1f0dtavsd14ncf17gbsy1cvoga4.jpeg" />

In [132]:
!ls ../input/sarcasm/

In [133]:
# some necessary imports
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
from matplotlib import pyplot as plt

In [134]:
train_df = pd.read_csv('../input/sarcasm/train-balanced-sarcasm.csv')

In [135]:
train_df.head()

In [136]:
train_df.info()

Some comments are missing, so we drop the corresponding rows.

In [137]:
train_df.dropna(subset=['comment'], inplace=True)

We notice that the dataset is indeed balanced

In [138]:
train_df['label'].value_counts()

We split data into training and validation parts.

In [139]:
text_train, text_value, y_train, y_value = \
        train_test_split(train_df['comment'], train_df['label'], random_state=17)

In [140]:
train_df.loc[train_df['label'] == 1, 'comment'].str.len().apply(np.log1p).hist(label='sarcastic')
train_df.loc[train_df['label'] == 0, 'comment'].str.len().apply(np.log1p).hist(label='normal')
plt.legend();

In [141]:
sub_df = train_df.groupby('subreddit')['label'].agg([np.size, np.mean, np.sum])
sub_df.sort_values(by='sum', ascending=False).head(10)

In [142]:
sub_df[sub_df['size'] > 1000].sort_values(by='mean', ascending=False).head(10)

In [143]:
sub_df = train_df.groupby('author')['label'].agg([np.size, np.mean, np.sum])
sub_df[sub_df['size'] > 300].sort_values(by='mean', ascending=False).head(10)

In [144]:
# build bigrams, put a limit on maximal number of features
# and minimal word frequency
tf_idf = TfidfVectorizer(ngram_range=(1, 2), max_features=50000, min_df=2)
# multinomial logistic regression a.k.a softmax classifier
logit = LogisticRegression(C=1, n_jobs=4, solver='lbfgs', 
                           random_state=17, verbose=1)
# sklearn's pipeline
my_pipeline = Pipeline([('tf_idf', tf_idf), 
                                 ('logit', logit)])

In [145]:
%%time
my_pipeline.fit(text_train, y_train)

In [146]:
%%time
y_pred = my_pipeline.predict(text_value)

In [147]:
y_pred

In [148]:
accuracy_score(y_value, y_pred)

In [149]:
# №3
import eli5
eli5.show_weights(estimator=tfidf_logit_pipeline.named_steps['logit'],
                  vec=tfidf_logit_pipeline.named_steps['tf_idf'])

In [ ]:
# №4

In [151]:
subreddits = train_df['subreddit']
train_subreddits, valid_subreddits = train_test_split(subreddits, random_state=17)
train_subreddits

In [152]:
valid_subreddits

In [153]:
tf_idf_texts = TfidfVectorizer(ngram_range=(1, 2), max_features=50000, min_df=2)
tf_idf_subreddits = TfidfVectorizer(ngram_range=(1, 1))

In [154]:
tf_idf_texts

In [155]:
tf_idf_subreddits

In [156]:
%%time
x_train = tf_idf_texts.fit_transform(text_train)
x_value = tf_idf_texts.transform(text_value)

In [157]:
x_train.shape, x_value.shape

In [158]:
%%time
x_train_sub = tf_idf_subreddits.fit_transform(train_subreddits)
x_value_sub = tf_idf_subreddits.transform(valid_subreddits)

In [159]:
x_train_sub

In [160]:
x_value_sub

In [161]:
x_train_sub.shape, x_value_sub.shape

In [163]:
from scipy.sparse import hstack
X_train = hstack([x_train, x_train_sub])
X_value = hstack([x_value, x_value_sub])

In [164]:
X_train

In [165]:
X_value

In [166]:
X_train.shape, X_value.shape

In [167]:
logit.fit(X_train, y_train)

In [171]:
%%time
y_pred = logit.predict(X_value)

In [172]:
y_pred

In [173]:
accuracy_score(y_value, y_pred)

## Tasks:
1. Analyze the dataset, make some plots. This [Kernel](https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-qiqc) might serve as an example
2. Build a Tf-Idf + logistic regression pipeline to predict sarcasm (`label`) based on the text of a comment on Reddit (`comment`).
3. Plot the words/bigrams which a most predictive of sarcasm (you can use [eli5](https://github.com/TeamHG-Memex/eli5) for that)
4. (optionally) add subreddits as new features to improve model performance. Apply here the Bag of Words approach, i.e. treat each subreddit as a new feature.

## Links:
  - Machine learning library [Scikit-learn](https://scikit-learn.org/stable/index.html) (a.k.a. sklearn)
  - Kernels on [logistic regression](https://www.kaggle.com/kashnitsky/topic-4-linear-models-part-2-classification) and its applications to [text classification](https://www.kaggle.com/kashnitsky/topic-4-linear-models-part-4-more-of-logit), also a [Kernel](https://www.kaggle.com/kashnitsky/topic-6-feature-engineering-and-feature-selection) on feature engineering and feature selection
  - [Kaggle Kernel](https://www.kaggle.com/abhishek/approaching-almost-any-nlp-problem-on-kaggle) "Approaching (Almost) Any NLP Problem on Kaggle"
  - [ELI5](https://github.com/TeamHG-Memex/eli5) to explain model predictions